In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore", message="The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.", category=FutureWarning)

In [8]:
DATA_PATH = '/kaggle/input/march-machine-learning-mania-2024/'


# 1. 生成男/女性的队伍数据

In [9]:
RSresultW_df = pd.read_csv(DATA_PATH+'WRegularSeasonDetailedResults.csv')
NCAresultW_df = pd.read_csv(DATA_PATH+'WNCAATourneyDetailedResults.csv')

In [10]:
RSresultW_df['ScoreDiff'] = RSresultW_df['WScore'] - RSresultW_df['LScore']
NCAresultW_df['ScoreDiff'] = NCAresultW_df['WScore'] - NCAresultW_df['LScore']
Wresult = pd.concat([RSresultW_df, NCAresultW_df]).reset_index(drop=True)

In [11]:
all_teams = pd.unique(pd.concat([Wresult['WTeamID'], Wresult['LTeamID']]))
print(f"The total number of teams is {len(all_teams)}.")

The total number of teams is 364.


In [12]:
years = Wresult['Season'].unique()

WTeamData = pd.DataFrame(columns=['TeamID', 'Season', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'ScoreVar', 
                                 'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk', 'WinPct'])

for year in years:
    # 逐年逐队分析
    year_data = Wresult[Wresult['Season'] == year]
    
    all_teams = pd.unique(pd.concat([year_data['WTeamID'], year_data['LTeamID']]))
    
    for team_id in all_teams:
        win_data = year_data[year_data['WTeamID'] == team_id]
        win_stats = win_data[['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk']]
        win_stats.columns = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk']
        
        loss_data = year_data[year_data['LTeamID'] == team_id]
        loss_stats = loss_data[['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LStl', 'LBlk']]
        loss_stats.columns = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'Stl', 'Blk']
        
        # 记录对手数据
        win_opponent_stats = win_data[['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LStl', 'LBlk']]
        win_opponent_stats.columns = ['OppScore','OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']
        loss_opponent_stats = loss_data[['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WStl', 'WBlk']]
        loss_opponent_stats.columns = ['OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']
        
        # 合并胜利和失败数据
        team_stats = pd.concat([win_stats, loss_stats], axis=0)
        opponent_stats = pd.concat([win_opponent_stats, loss_opponent_stats], axis=0)
        
        avg_stats = team_stats.mean(axis=0)
        
        var_score = team_stats['Score'].var()
        
        total_games = len(win_data) + len(loss_data)
        win_pct = len(win_data) / total_games if total_games > 0 else 0
        
        avg_stats['TeamID'] = team_id
        avg_stats['Season'] = year
        avg_stats['ScoreVar'] = var_score
        avg_stats['WinPct'] = win_pct
        
        # 添加对手统计数据列
        avg_stats = pd.concat([avg_stats, opponent_stats.mean(axis=0)], axis=0)
        
        WTeamData = WTeamData.append(avg_stats, ignore_index=True)

WTeamData = WTeamData[['TeamID', 'Season', 'Score', 'WinPct', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'ScoreVar', 
                     'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']]

WTeamData['TeamID'] = WTeamData['TeamID'].astype(Wresult['WTeamID'].dtype)
WTeamData['Season'] = WTeamData['Season'].astype(Wresult['Season'].dtype)

In [13]:
WTeamData

,TeamID,Season,Score,WinPct,FGM,FGA,FGM3,FGA3,FTM,FTA,...,ScoreVar,OppScore,OppFGM,OppFGA,OppFGM3,OppFGA3,OppFTM,OppFTA,OppStl,OppBlk
0,3103,2010,62.233333,0.566667,22.233333,55.266667,3.933333,12.466667,13.833333,19.600000,...,87.219540,59.633333,20.966667,54.966667,4.833333,15.166667,12.866667,18.833333,9.233333,3.800000
1,3104,2010,64.137931,0.379310,24.724138,62.103448,4.344828,14.724138,10.344828,16.793103,...,171.408867,67.000000,23.896552,60.137931,5.103448,16.517241,14.103448,21.965517,8.758621,3.586207
2,3110,2010,60.428571,0.619048,21.571429,53.904762,4.666667,15.285714,12.619048,16.809524,...,132.657143,54.380952,20.000000,52.904762,4.095238,14.190476,10.285714,15.000000,5.714286,2.666667
3,3111,2010,66.419355,0.612903,25.774194,59.483871,4.290323,12.387097,10.580645,16.612903,...,92.651613,63.838710,21.516129,59.193548,6.129032,18.645161,14.677419,20.548387,9.741935,3.516129
4,3119,2010,52.291667,0.333333,19.166667,53.541667,4.625000,16.916667,9.333333,13.875000,...,90.737319,58.750000,20.750000,54.750000,4.541667,13.166667,12.708333,18.458333,7.875000,3.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,3141,2024,60.583333,0.208333,22.333333,58.416667,5.291667,19.000000,10.625000,16.333333,...,59.297101,72.125000,26.000000,64.041667,7.458333,21.541667,12.666667,17.291667,10.291667,2.958333
5236,3476,2024,51.846154,0.153846,18.692308,55.576923,5.423077,19.923077,9.038462,12.961538,...,110.055385,68.230769,26.538462,61.500000,5.153846,17.500000,10.000000,14.846154,11.115385,3.423077
5237,3214,2024,55.041667,0.083333,19.416667,56.500000,3.000000,12.375000,13.208333,20.416667,...,84.389493,69.166667,23.083333,56.541667,6.541667,20.583333,16.458333,22.500000,8.541667,3.416667
5238,3152,2024,57.192308,0.000000,21.538462,59.961538,2.461538,10.769231,11.653846,16.153846,...,211.681538,84.769231,31.038462,67.846154,6.730769,21.153846,15.961538,23.538462,11.615385,2.423077


In [14]:
MRSresultW_df = pd.read_csv(DATA_PATH+'MRegularSeasonDetailedResults.csv')
MNCAresultW_df = pd.read_csv(DATA_PATH+'MNCAATourneyDetailedResults.csv')

In [15]:
MRSresultW_df['ScoreDiff'] = MRSresultW_df['WScore'] - MRSresultW_df['LScore']
MNCAresultW_df['ScoreDiff'] = MNCAresultW_df['WScore'] - MNCAresultW_df['LScore']
Mresult = pd.concat([MRSresultW_df, MNCAresultW_df]).reset_index(drop=True)

In [16]:
Mall_teams = pd.unique(pd.concat([Mresult['WTeamID'], Mresult['LTeamID']]))
print(f"The total number of teams is {len(all_teams)}.")

The total number of teams is 360.


In [17]:
years = Mresult['Season'].unique()

MTeamData = pd.DataFrame(columns=['TeamID', 'Season', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'ScoreVar', 
                                 'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk', 'WinPct'])

for year in years:
    # 逐年逐队分析
    year_data = Mresult[Mresult['Season'] == year]
    
    Mall_teams = pd.unique(pd.concat([year_data['WTeamID'], year_data['LTeamID']]))
    
    for team_id in all_teams:
        win_data = year_data[year_data['WTeamID'] == team_id]
        win_stats = win_data[['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk']]
        win_stats.columns = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk']
        
        loss_data = year_data[year_data['LTeamID'] == team_id]
        loss_stats = loss_data[['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LStl', 'LBlk']]
        loss_stats.columns = ['Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'Stl', 'Blk']
        
        # 记录对手数据
        win_opponent_stats = win_data[['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LStl', 'LBlk']]
        win_opponent_stats.columns = ['OppScore','OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']
        loss_opponent_stats = loss_data[['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WStl', 'WBlk']]
        loss_opponent_stats.columns = ['OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']
        
        # 合并胜利和失败数据
        team_stats = pd.concat([win_stats, loss_stats], axis=0)
        opponent_stats = pd.concat([win_opponent_stats, loss_opponent_stats], axis=0)
        
        avg_stats = team_stats.mean(axis=0)
        
        var_score = team_stats['Score'].var()
        
        total_games = len(win_data) + len(loss_data)
        win_pct = len(win_data) / total_games if total_games > 0 else 0
        
        avg_stats['TeamID'] = team_id
        avg_stats['Season'] = year
        avg_stats['ScoreVar'] = var_score
        avg_stats['WinPct'] = win_pct
        
        # 添加对手统计数据列
        avg_stats = pd.concat([avg_stats, opponent_stats.mean(axis=0)], axis=0)
        
        MTeamData = MTeamData.append(avg_stats, ignore_index=True)

MTeamData = MTeamData[['TeamID', 'Season', 'Score', 'WinPct', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'ScoreVar', 
                     'OppScore', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppStl', 'OppBlk']]

MTeamData['TeamID'] = MTeamData['TeamID'].astype(Mresult['WTeamID'].dtype)
MTeamData['Season'] = MTeamData['Season'].astype(Mresult['Season'].dtype)



In [18]:
MTeamData

,TeamID,Season,Score,WinPct,FGM,FGA,FGM3,FGA3,FTM,FTA,...,ScoreVar,OppScore,OppFGM,OppFGA,OppFGM3,OppFGA3,OppFTM,OppFTA,OppStl,OppBlk
0,3102,2003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3104,2003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3107,2003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3110,2003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3111,2003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,3141,2024,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7916,3476,2024,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7917,3214,2024,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7918,3152,2024,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. 生成适用于模型的dataset
### 2.1 只取有详细记录的2010年迄今数据，并打乱比赛结果(csv中分为WTeam和LTeam，没有明确标记结果)

In [19]:
RSresultW = pd.read_csv(DATA_PATH+'WRegularSeasonCompactResults.csv')
RSresultW = RSresultW[RSresultW['Season']>=2010]
RSresultW_df = RSresultW.drop_duplicates(subset=['Season', 'WTeamID', 'LTeamID'])

In [20]:
NCAAresultW = pd.read_csv(DATA_PATH+'WNCAATourneyCompactResults.csv')
NCAAresultW = NCAAresultW[NCAAresultW['Season']>=2010]
NCAAresultW_df = NCAAresultW.drop_duplicates(subset=['Season', 'WTeamID', 'LTeamID'])

In [21]:
MRSresultW = pd.read_csv(DATA_PATH+'MRegularSeasonCompactResults.csv')
MRSresultW = MRSresultW[MRSresultW['Season']>=2010]
MRSresultW_df = MRSresultW.drop_duplicates(subset=['Season', 'WTeamID', 'LTeamID'])

In [22]:
MNCAAresultW = pd.read_csv(DATA_PATH+'MNCAATourneyCompactResults.csv')
MNCAAresultW = MNCAAresultW[MNCAAresultW['Season']>=2010]
MNCAAresultW_df = MNCAAresultW.drop_duplicates(subset=['Season', 'WTeamID', 'LTeamID'])

In [23]:
# 定义函数以随机交换队伍和分数
def shuffle_teams(row):
    if np.random.rand() > 0.5:
        # 如果随机数大于0.5，交换胜利和失败队伍的ID及其分数
        return pd.Series({'Year':row['Season'], 'Team1': row['LTeamID'], 'Score1': row['LScore'], 
                          'Team2': row['WTeamID'], 'Score2': row['WScore'], 
                          'Result': 1 if row['LScore'] > row['WScore'] else (2 if row['LScore'] < row['WScore'] else 0)})
    else:
        return pd.Series({'Year':row['Season'], 'Team1': row['WTeamID'], 'Score1': row['WScore'], 
                          'Team2': row['LTeamID'], 'Score2': row['LScore'], 
                          'Result': 1 if row['WScore'] > row['LScore'] else (2 if row['WScore'] < row['LScore'] else 0)})


In [24]:
RSresultW_df2 = RSresultW_df.apply(shuffle_teams, axis=1)
RSresultW_df2 = RSresultW_df2.drop(['Score1', 'Score2'], axis=1)
RSresultW_df2['TeamPair'] = RSresultW_df2.apply(lambda row: tuple(sorted([row['Team1'], row['Team2']])), axis=1)
RSresultW_df3 = RSresultW_df2.drop_duplicates(subset=['Year','TeamPair'])
# RSresultW_df3 = RSresultW_df3.drop(columns='TeamPair')
RSresultW_df3

,Year,Team1,Team2,Result,TeamPair
55101,2010,3103,3237,1,"(3103, 3237)"
55102,2010,3104,3399,1,"(3104, 3399)"
55103,2010,3224,3110,2,"(3110, 3224)"
55104,2010,3267,3111,2,"(3111, 3267)"
55105,2010,3119,3447,1,"(3119, 3447)"
...,...,...,...,...,...
130884,2024,3374,3455,1,"(3374, 3455)"
130886,2024,3247,3387,2,"(3247, 3387)"
130889,2024,3111,3407,2,"(3111, 3407)"
130892,2024,3433,3348,1,"(3348, 3433)"


In [25]:
MRSresultW_df2 = MRSresultW_df.apply(shuffle_teams, axis=1)
MRSresultW_df2 = MRSresultW_df2.drop(['Score1', 'Score2'], axis=1)
MRSresultW_df2['TeamPair'] = MRSresultW_df2.apply(lambda row: tuple(sorted([row['Team1'], row['Team2']])), axis=1)
MRSresultW_df3 = MRSresultW_df2.drop_duplicates(subset=['Year','TeamPair'])
# RSresultW_df3 = RSresultW_df3.drop(columns='TeamPair')
MRSresultW_df3

,Year,Team1,Team2,Result,TeamPair
108122,2010,1293,1143,2,"(1143, 1293)"
108123,2010,1198,1314,2,"(1198, 1314)"
108124,2010,1326,1108,1,"(1108, 1326)"
108125,2010,1393,1107,1,"(1107, 1393)"
108126,2010,1178,1143,2,"(1143, 1178)"
...,...,...,...,...,...
186538,2024,1120,1397,2,"(1120, 1397)"
186540,2024,1329,1416,2,"(1329, 1416)"
186543,2024,1433,1348,1,"(1348, 1433)"
186544,2024,1130,1438,2,"(1130, 1438)"


In [26]:
NCAAresultW_df2 = NCAAresultW_df.apply(shuffle_teams, axis=1)
NCAAresultW_df2 = NCAAresultW_df2.drop(['Score1', 'Score2'], axis=1)
NCAAresultW_df2['TeamPair'] = NCAAresultW_df2.apply(lambda row: tuple(sorted([row['Team1'], row['Team2']])), axis=1)
NCAAresultW_df2 = NCAAresultW_df2.drop_duplicates(subset=['Year','TeamPair'])
NCAAresultW_df2

,Year,Team1,Team2,Result,TeamPair
756,2010,3201,3124,2,"(3124, 3201)"
757,2010,3395,3173,2,"(3173, 3395)"
758,2010,3214,3181,2,"(3181, 3214)"
759,2010,3199,3256,1,"(3199, 3256)"
760,2010,3207,3265,1,"(3207, 3265)"
...,...,...,...,...,...
1578,2023,3268,3376,2,"(3268, 3376)"
1579,2023,3439,3326,1,"(3326, 3439)"
1580,2023,3234,3376,1,"(3234, 3376)"
1581,2023,3439,3261,2,"(3261, 3439)"


In [27]:
MNCAAresultW_df2 = MNCAAresultW_df.apply(shuffle_teams, axis=1)
MNCAAresultW_df2 = MNCAAresultW_df2.drop(['Score1', 'Score2'], axis=1)
MNCAAresultW_df2['TeamPair'] = MNCAAresultW_df2.apply(lambda row: tuple(sorted([row['Team1'], row['Team2']])), axis=1)
MNCAAresultW_df2 = MNCAAresultW_df2.drop_duplicates(subset=['Year','TeamPair'])
MNCAAresultW_df2

,Year,Team1,Team2,Result,TeamPair
1584,2010,1115,1457,1,"(1115, 1457)"
1585,2010,1358,1124,2,"(1124, 1358)"
1586,2010,1431,1139,2,"(1139, 1431)"
1587,2010,1140,1196,1,"(1140, 1196)"
1588,2010,1250,1242,2,"(1242, 1250)"
...,...,...,...,...,...
2446,2023,1274,1400,1,"(1274, 1400)"
2447,2023,1166,1361,2,"(1166, 1361)"
2448,2023,1163,1274,1,"(1163, 1274)"
2449,2023,1194,1361,2,"(1194, 1361)"


In [28]:
Wresult_df = pd.concat([RSresultW_df3, NCAAresultW_df2], ignore_index=True)
Wresult_df = Wresult_df.drop_duplicates(subset=['Year','TeamPair'],keep='last')
Wresult_df = Wresult_df.drop(columns='TeamPair')
Wresult_df

,Year,Team1,Team2,Result
0,2010,3103,3237,1
1,2010,3104,3399,1
2,2010,3224,3110,2
3,2010,3267,3111,2
4,2010,3119,3447,1
...,...,...,...,...
54990,2023,3268,3376,2
54991,2023,3439,3326,1
54992,2023,3234,3376,1
54993,2023,3439,3261,2


In [29]:
MWresult_df = pd.concat([MRSresultW_df3, MNCAAresultW_df2], ignore_index=True)
MWresult_df = MWresult_df.drop_duplicates(subset=['Year','TeamPair'],keep='last')
MWresult_df = MWresult_df.drop(columns='TeamPair')
MWresult_df

,Year,Team1,Team2,Result
0,2010,1293,1143,2
1,2010,1198,1314,2
2,2010,1326,1108,1
3,2010,1393,1107,1
4,2010,1178,1143,2
...,...,...,...,...
56493,2023,1274,1400,1
56494,2023,1166,1361,2
56495,2023,1163,1274,1
56496,2023,1194,1361,2


In [30]:
merged_df1 = pd.merge(Wresult_df, WTeamData, left_on=['Year', 'Team1'], right_on=['Season', 'TeamID'], how='left')
merged_df1 = merged_df1.rename(columns={
    'Score': 'Score1', 
    'FGM': 'FGM1', 
    'FGA': 'FGA1', 
    'FGM3': 'FGM31', 
    'FGA3': 'FGA31', 
    'FTM': 'FTM1', 
    'FTA': 'FTA1', 
    'OR': 'OR1', 
    'DR': 'DR1', 
    'Ast': 'Ast1', 
    'TO': 'TO1', 
    'Stl': 'Stl1', 
    'Blk': 'Blk1', 
    'ScoreVar': 'ScoreVar1', 
    'OppScore': 'OppScore1', 
    'OppFGM': 'OppFGM1', 
    'OppFGA': 'OppFGA1', 
    'OppFGM3': 'OppFGM31', 
    'OppFGA3': 'OppFGA31', 
    'OppFTM': 'OppFTM1', 
    'OppFTA': 'OppFTA1', 
    'OppStl': 'OppStl1', 
    'OppBlk': 'OppBlk1', 
    'WinPct': 'WinPct1'
})


In [32]:
Mmerged_df1 = pd.merge(MWresult_df, MTeamData, left_on=['Year', 'Team1'], right_on=['Season', 'TeamID'], how='left')
Mmerged_df1 = Mmerged_df1.rename(columns={
    'Score': 'Score1', 
    'FGM': 'FGM1', 
    'FGA': 'FGA1', 
    'FGM3': 'FGM31', 
    'FGA3': 'FGA31', 
    'FTM': 'FTM1', 
    'FTA': 'FTA1', 
    'OR': 'OR1', 
    'DR': 'DR1', 
    'Ast': 'Ast1', 
    'TO': 'TO1', 
    'Stl': 'Stl1', 
    'Blk': 'Blk1', 
    'ScoreVar': 'ScoreVar1', 
    'OppScore': 'OppScore1', 
    'OppFGM': 'OppFGM1', 
    'OppFGA': 'OppFGA1', 
    'OppFGM3': 'OppFGM31', 
    'OppFGA3': 'OppFGA31', 
    'OppFTM': 'OppFTM1', 
    'OppFTA': 'OppFTA1', 
    'OppStl': 'OppStl1', 
    'OppBlk': 'OppBlk1', 
    'WinPct': 'WinPct1'
})


In [33]:
merged_df2 = pd.merge(Wresult_df, WTeamData, left_on=['Year', 'Team2'], right_on=['Season', 'TeamID'], how='left')
merged_df2 = merged_df2.rename(columns={
    'Score': 'Score2', 
    'FGM': 'FGM2', 
    'FGA': 'FGA2', 
    'FGM3': 'FGM32', 
    'FGA3': 'FGA32', 
    'FTM': 'FTM2', 
    'FTA': 'FTA2', 
    'OR': 'OR2', 
    'DR': 'DR2', 
    'Ast': 'Ast2', 
    'TO': 'TO2', 
    'Stl': 'Stl2', 
    'Blk': 'Blk2', 
    'ScoreVar': 'ScoreVar2', 
    'OppScore': 'OppScore2', 
    'OppFGM': 'OppFGM2', 
    'OppFGA': 'OppFGA2', 
    'OppFGM3': 'OppFGM32', 
    'OppFGA3': 'OppFGA32', 
    'OppFTM': 'OppFTM2', 
    'OppFTA': 'OppFTA2', 
    'OppStl': 'OppStl2', 
    'OppBlk': 'OppBlk2', 
    'WinPct': 'WinPct2'
})

In [34]:
Mmerged_df2 = pd.merge(MWresult_df, MTeamData, left_on=['Year', 'Team2'], right_on=['Season', 'TeamID'], how='left')
Mmerged_df2 = Mmerged_df2.rename(columns={
    'Score': 'Score2', 
    'FGM': 'FGM2', 
    'FGA': 'FGA2', 
    'FGM3': 'FGM32', 
    'FGA3': 'FGA32', 
    'FTM': 'FTM2', 
    'FTA': 'FTA2', 
    'OR': 'OR2', 
    'DR': 'DR2', 
    'Ast': 'Ast2', 
    'TO': 'TO2', 
    'Stl': 'Stl2', 
    'Blk': 'Blk2', 
    'ScoreVar': 'ScoreVar2', 
    'OppScore': 'OppScore2', 
    'OppFGM': 'OppFGM2', 
    'OppFGA': 'OppFGA2', 
    'OppFGM3': 'OppFGM32', 
    'OppFGA3': 'OppFGA32', 
    'OppFTM': 'OppFTM2', 
    'OppFTA': 'OppFTA2', 
    'OppStl': 'OppStl2', 
    'OppBlk': 'OppBlk2', 
    'WinPct': 'WinPct2'
})

In [35]:
# 合并merged_df1和merged_df2
final_df = pd.merge(merged_df1, merged_df2, on=['Year', 'Team1', 'Team2', 'Result'], how='left')

# 选择所需的列
columns_to_keep = [
    'Year', 'Result','Score1', 'FGM1', 'FGA1', 'FGM31', 'FGA31', 'FTM1', 'FTA1', 'OR1', 'DR1', 
    'Ast1', 'TO1', 'Stl1', 'Blk1', 'ScoreVar1', 'OppScore1', 'OppFGM1', 'OppFGA1', 
    'OppFGM31', 'OppFGA31', 'OppFTM1', 'OppFTA1', 'OppStl1', 'OppBlk1', 'WinPct1',
    'Score2', 'FGM2', 'FGA2', 'FGM32', 'FGA32', 'FTM2', 'FTA2', 'OR2', 'DR2', 
    'Ast2', 'TO2', 'Stl2', 'Blk2', 'ScoreVar2', 'OppScore2', 'OppFGM2', 'OppFGA2', 
    'OppFGM32', 'OppFGA32', 'OppFTM2', 'OppFTA2', 'OppStl2', 'OppBlk2', 'WinPct2'
]

Wdataset = final_df[columns_to_keep].dropna()

In [36]:
# 合并merged_df1和merged_df2
Mfinal_df = pd.merge(Mmerged_df1, Mmerged_df2, on=['Year', 'Team1', 'Team2', 'Result'], how='left')

# 选择所需的列
columns_to_keep = [
    'Year', 'Result','Score1', 'FGM1', 'FGA1', 'FGM31', 'FGA31', 'FTM1', 'FTA1', 'OR1', 'DR1', 
    'Ast1', 'TO1', 'Stl1', 'Blk1', 'ScoreVar1', 'OppScore1', 'OppFGM1', 'OppFGA1', 
    'OppFGM31', 'OppFGA31', 'OppFTM1', 'OppFTA1', 'OppStl1', 'OppBlk1', 'WinPct1',
    'Score2', 'FGM2', 'FGA2', 'FGM32', 'FGA32', 'FTM2', 'FTA2', 'OR2', 'DR2', 
    'Ast2', 'TO2', 'Stl2', 'Blk2', 'ScoreVar2', 'OppScore2', 'OppFGM2', 'OppFGA2', 
    'OppFGM32', 'OppFGA32', 'OppFTM2', 'OppFTA2', 'OppStl2', 'OppBlk2', 'WinPct2'
]

MWdataset = Mfinal_df[columns_to_keep].dropna()